## Data


Skrypt konwertujący rozgrywkę szachową zapisaną w formacie PGN, na listę pozycji szachowych zapisanych w formacie FEN

In [ ]:
#!/usr/bin/python3
# Konwertuje plik z partią szachową w notacji PGN do listy FEN-ów (po każdym ruchu)
# Wymagania: python-chess

import chess
import chess.pgn

with open('pgngames.txt',"r") as input_f, open('1000first.txt',"w") as output_f:
    game=chess.pgn.read_game(input_f)
    board=chess.Board()

    for move in game.mainline_moves():
        board.push(move)
        output_f.write(board.epd()+'\n')

ERROR:chess.pgn:illegal san: '1b1' in rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1 while parsing <Game at 0x79bc792ab100 ('?' vs. '?', '????.??.??' at '?')>


Funkcja przyjmująca pozycje w formacie FEN i oceniajaca ją przy użyuciu stockfisha

In [6]:
from stockfish import Stockfish
import typing as t
import numpy as np

def fen_convert(fen_positions: t.List[str]) -> t.List[str]:
    # Add your path to stockfish
    path = ""
    stockfish = Stockfish(path, depth=25, parameters={"Hash": 2048} )
    evaluations = np.zeros(len(fen_positions))
    for index, fen in enumerate(fen_positions):
        stockfish.set_fen_position(fen)
        evaluations[index] = stockfish.get_evaluation()['value']
    return evaluations

TEST_FEN = [
    "r1bk3r/p2pBpNp/n4n2/1p1NP2P/6P1/3P4/P1P1K3/q5b1",
    " 8/8/8/4p1K1/2k1P3/8/8/8 b - - 0 1",
    "4k2r/6r1/8/8/8/8/3R4/R3K3 w Qk - 0 1",
    "rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1",
    "8/5k2/3p4/1p1Pp2p/pP2Pp1P/P4P1K/8/8 b - - 99 50",
    "r1b1k2r/pp3ppp/5b2/K2p1n2/2N5/4nN2/Pp4qP/8 w kq - 8 21"
    ]

print(fen_convert(TEST_FEN))

[    0. -3500.   426.    18.     0.    -4.]


Stockfish potrafi oceniać pozycję bez ograniczeń. By narzucić ramy zwrócony przez stockfisha wynik przepuszczamy przez sigmoid, co sprowadza nasz wynik do przedziału [-1, 1]

In [7]:

import numpy as np


def sigmoid(p):
    return 1/(1+np.exp(-p*0.00368208))


def fen_convert(fen_positions):
    # Add your path to stockfish
    path = ""
    stockfish = Stockfish(r'C:\Users\bemic\Desktop\IT-Projekty\ML-Project\ML-Chess-Eval\stockfish\stockfish-windows-x86-64-modern.exe', depth=25, parameters={"Hash": 2048} )
    evaluations = np.zeros(len(fen_positions))
    stockfish.update_engine_parameters({'UCI_LimitStrength': False})
    for index, fen in enumerate(fen_positions):
        stockfish.set_fen_position(fen)
        evaluations[index] = stockfish.get_evaluation()['value']
        eval = sigmoid(stockfish.get_evaluation()['value'])
        evaluations[index] = eval
    return evaluations

TEST_FEN = [
    "r1bk3r/p2pBpNp/n4n2/1p1NP2P/6P1/3P4/P1P1K3/q5b1",
    " 8/8/8/4p1K1/2k1P3/8/8/8 b - - 0 1",
    "4k2r/6r1/8/8/8/8/3R4/R3K3 w Qk - 0 1",
    "rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1",
    "8/5k2/3p4/1p1Pp2p/pP2Pp1P/P4P1K/8/8 b - - 99 50",
    "r1b1k2r/pp3ppp/5b2/K2p1n2/2N5/4nN2/Pp4qP/8 w kq - 8 21"
    ]

print(fen_convert(TEST_FEN))

[5.00000000e-01 2.05861210e-06 8.79955125e-01 5.18402084e-01
 5.00000000e-01 4.96317987e-01]


Skrypt czytający z pliku pozycje szachowe w postaci FEN, dopisujący do nich ocene pozycji zaproponowaną przez Stockfisha, a następnie zapisująca jako DataFrame

In [ ]:
import sys
import numpy as np
import pandas as pd


with open('1000first.txt',"r") as input_f:
    Arr = []
    line = input_f.read().split('\n')
    y_value = fen_convert(line)
    x_value = line
    d = {'FEN': x_value, 'Eval': y_value}
    dataset = pd.DataFrame(data=d)
    dataset.to_csv('data_after_parse.txt', sep='\t', index=False)

 Funkcja zamieniająca pozycję w formacie FEN na format bardziej przypominający obrazek. Obrazek składa się z 67 wymiarów, pierwsze 64 wymiary odpowiadają polom szachownicy i tym jaka znajduje się na nim figura. 65 wymiar opisuje czyja jest aktualnie tura (w szachach ma to istotny wpływ na ocenę pozycji). 66 wymiar przechowuje informacje na temat możliwych roszad, a 67 na temat ruchów en-passant

 Dane na tym etapie nie są już zapisywane do plików, a rozdzielane na X_Train (pozycja), Y_Train(ocena)

In [ ]:
with open("data_after_parse.txt","r") as input_f:
    WHITE='PNBRQK'
    BLACK='pnbrqk'
    TURN={
        'w': 0,
        'b': 1
    }
    CASTLING={
        '-': 0,
        'K': 1,
        'Q': 2,
        'k': 3,
        'q': 4,
        'KQ': 5,
        'Kk': 6,
        'Kq': 7,
        'Qk': 8,
        'Qq': 9,
        'kq': 10,
        'KQk': 11,
        'KQq': 12,
        'Kkq': 13,
        'Qkq': 14,
        'KQkq': 15
    }
    EN_PASSANT={
        '-': 0,
        'a3': 1,
        'b3': 2,
        'c3': 3,
        'd3': 4,
        'e3': 5,
        'f3': 6,
        'g3': 7,
        'h3': 8,
        'a6': 9,
        'b6': 10,
        'c6': 11,
        'd6': 12,
        'e6': 13,
        'f6': 14,
        'g6': 15,
        'h6': 16,
    }
    line = input_f.read().split('\n')
    X_old_train = []
    Y_old_train = []
    for idx, fen in enumerate(line):
        fen = fen.replace('\t', ' ')
        pos, turn, castling, en_passant, evaluation = fen.split(" ")
        Board = []
        counter=0
        for line in pos.split('/'):
            for i in line:
                if i in WHITE:
                    Board.append(WHITE.index(i)+1)
                    counter+=1
                elif i in BLACK:
                    Board.append(BLACK.index(i)+7)
                    counter+=1
                else:
                    for lmap in range(int(i)):
                        Board.append(0)
        Board.append(TURN[turn])
        Board.append(CASTLING[castling])
        Board.append(EN_PASSANT[en_passant])
        X_old_train.append(Board)
        Y_old_train.append(evaluation)

In [ ]:
print(X_old_train[0:10])
print(Y_old_train[0:10])

[[10, 8, 9, 11, 12, 9, 8, 10, 7, 7, 7, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 4, 2, 3, 5, 6, 3, 2, 4, 1, 15, 0], [10, 8, 9, 11, 12, 9, 8, 10, 7, 7, 0, 7, 7, 7, 7, 7, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 4, 2, 3, 5, 6, 3, 2, 4, 0, 15, 0], [10, 8, 9, 11, 12, 9, 8, 10, 7, 7, 0, 7, 7, 7, 7, 7, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 4, 2, 3, 5, 6, 3, 0, 4, 1, 15, 0], [10, 8, 9, 11, 12, 9, 8, 10, 7, 7, 0, 0, 7, 7, 7, 7, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 4, 2, 3, 5, 6, 3, 0, 4, 0, 15, 0], [10, 8, 9, 11, 12, 9, 8, 10, 7, 7, 0, 0, 7, 7, 7, 7, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 1, 1, 5, 1, 

Zaproponowany wyżej format nie sprawdził się dobrze przy pierwszych próbach użycia drzew decyzyjnych, postanowiliśmy zareprezentować szachownice inaczej. Tym razem szachownica będzie reprezentowana przy pomocy 789 wymiarów

        Wymiary 0-767 <- jakie figury są na szachownicy i gdzie
        (pierwsze 64 wymiary: białe piony; reszta wedle poprzedniej konwencji)
        Wymiar 768 <- czyj ruch
        Wymiary 769-772 <- roszady
        Wymiary 773-788 <- bicia w przelocie

In [ ]:
import itertools
import numpy as np
import pandas as pd
import sys
import os

with open("data_after_parse.txt","r") as input_f:
    WHITE='PNBRQK'
    BLACK='pnbrqk'
    EMPTY='12345678'
    TURN={
        'w': 0,
        'b': 1
    }

    EN_PASSANT={
        '-': 0,
        'a3': 1,
        'b3': 2,
        'c3': 3,
        'd3': 4,
        'e3': 5,
        'f3': 6,
        'g3': 7,
        'h3': 8,
        'a6': 9,
        'b6': 10,
        'c6': 11,
        'd6': 12,
        'e6': 13,
        'f6': 14,
        'g6': 15,
        'h6': 16,
    }

    line = input_f.read().split('\n')

    X_train = np.zeros(len(line), dtype=object)
    Y_train = np.zeros(len(line))

    for idx, fen in enumerate(line):
        fen = fen.replace('\t', ' ')
        # print(fen.split(" "))
        pos, turn, castling, en_passant, evaluation = fen.split(" ")
        Board = np.zeros(789, dtype=int) # 768, 1, 4, 16
        counter=0

        # Reprezentacja jest w 789 wymiarach
        # Wymiary 0-767 <- jakie figury są na szachownicy i gdzie
        # (pierwsze 64 wymiary: białe piony; reszta wedle konwencji na górze pliku)
        # Wymiar 768 <- czyj ruch
        # Wymiary 769-772 <- roszady
        # Wymiary 773-788 <- bicia w przelocie

        for line in pos.split('/'):
            for i in line:
                if i in WHITE:
                    n=WHITE.index(i)
                    Board[64*n+counter]=1
                    counter+=1
                elif i in BLACK:
                    n=BLACK.index(i)
                    Board[64*(6+n)+counter]=1
                    counter+=1
                else:
                    counter+=int(i)

        Board[768]=TURN[turn]

        for i,s in enumerate("KkQq"):
            if s in castling:
                Board[769+i]=1

        if en_passant!='-':
            Board[772+EN_PASSANT[en_passant]]=1

        X_train[idx] = Board
        Y_train[idx] = evaluation

In [ ]:
print(X_train[0:10])
print(Y_train[0:10])

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## Algorithms

Przy zwiększeniu wymiarów postanowiliśmy sprawdzić jak dużo informacji stracimy wykonując redukcje przy pomocy PCA

In [ ]:
from sklearn.decomposition import PCA

x_PCA = list(X_train)
pca = PCA(n_components=789)
X_PCA = pca.fit_transform(x_PCA)
print(pca.explained_variance_ratio_.cumsum().round(6))

[0.188584 0.233805 0.259544 0.281774 0.301425 0.320547 0.339474 0.35564
 0.370852 0.385413 0.399761 0.413438 0.426627 0.439111 0.451117 0.462724
 0.473778 0.484665 0.495164 0.505365 0.515032 0.524091 0.532735 0.540813
 0.548782 0.556255 0.563648 0.570811 0.577776 0.584582 0.591278 0.597642
 0.603811 0.60986  0.615557 0.621196 0.626604 0.631883 0.63709  0.642197
 0.6472   0.652121 0.656822 0.661462 0.665896 0.670239 0.674532 0.678803
 0.683022 0.687133 0.691151 0.695083 0.699002 0.702844 0.70665  0.71032
 0.713975 0.71757  0.721066 0.724428 0.72772  0.730976 0.734064 0.73709
 0.74008  0.742997 0.745876 0.748688 0.751473 0.754234 0.756911 0.759573
 0.762196 0.76475  0.767246 0.76973  0.772197 0.774595 0.776929 0.779233
 0.781512 0.783778 0.786032 0.788182 0.790301 0.792404 0.794491 0.796511
 0.798522 0.800525 0.802516 0.80449  0.806429 0.808336 0.810167 0.811992
 0.813775 0.815534 0.81725  0.818938 0.820606 0.822251 0.823879 0.825501
 0.827109 0.828704 0.83025  0.831777 0.83329  0.834784

In [ ]:
x_pca = list(X_train)
pca = PCA(n_components=789)
X_PCA = pca.fit_transform(x_pca)
treshold=0.8
for i, summm in enumerate(pca.explained_variance_ratio_.cumsum().round(6)):
    if summm > treshold:
        print(i, summm)
        treshold += 0.05

89 0.800525
121 0.851073
176 0.900087
304 0.950155


In [ ]:
# Boosted trees z sklearna na naszych szachach

import itertools
import numpy as np
import scipy.stats as ss
import pandas as pd
import sklearn.ensemble
import sys
import os

with open("data_after_parse.txt","r") as input_f:
    WHITE='PNBRQK'
    BLACK='pnbrqk'
    EMPTY='12345678'
    TURN={
        'w': 0,
        'b': 1
    }

    EN_PASSANT={
        '-': 0,
        'a3': 1,
        'b3': 2,
        'c3': 3,
        'd3': 4,
        'e3': 5,
        'f3': 6,
        'g3': 7,
        'h3': 8,
        'a6': 9,
        'b6': 10,
        'c6': 11,
        'd6': 12,
        'e6': 13,
        'f6': 14,
        'g6': 15,
        'h6': 16,
    }

    line = input_f.read().split('\n')

    X_train = np.zeros(len(line), dtype=object)
    Y_train = np.zeros(len(line))

    for idx, fen in enumerate(line):
        fen = fen.replace('\t', ' ')
        # print(fen.split(" "))
        pos, turn, castling, en_passant, evaluation = fen.split(" ")
        Board = np.zeros(789, dtype=int) # 768, 1, 4, 16
        counter=0

        for line in pos.split('/'):
            for i in line:
                if i in WHITE:
                    n=WHITE.index(i)
                    Board[64*n+counter]=1
                    counter+=1
                elif i in BLACK:
                    n=BLACK.index(i)
                    Board[64*(6+n)+counter]=1
                    counter+=1
                else:
                    counter+=int(i)

        Board[768]=TURN[turn]

        for i,s in enumerate("KkQq"):
            if s in castling:
                Board[769+i]=1

        if en_passant!='-':
            Board[772+EN_PASSANT[en_passant]]=1

        X_train[idx] = Board
        Y_train[idx] = evaluation

BoostTrees = sklearn.ensemble.HistGradientBoostingRegressor(max_iter=50000)

BoostTrees.fit(list(X_train), Y_train)
print(f"Accuracy: {BoostTrees.score(list(X_train),Y_train)}")

Accuracy: 0.8877022831193467


Zmieniliśmy podział na zbiór treningowy/testowy. Teraz zamiast sztywnego cięcia podział będzie się odbywał względem parti. Do tego wyliczamy ile zajęła każda z partii

In [ ]:

import sys
import chess
import chess.pgn

with open("data_after_parse.txt", 'r') as input_f, open('numbers.txt', 'w') as output_f:
    for i in range(1,213769420):
        game=chess.pgn.read_game(input_f)

        if game is None: continue
        board=chess.Board()

        counter=0
        for move in game.mainline_moves():
            board.push(move)
            counter+=1

        output_f.write(f'{counter}\n')

Spróbowaliśmy wykorzystać xgboost. Wyniki były zadowalające

In [ ]:
# Boosted trees z sklearna na naszych szachach

import itertools
import numpy as np
import scipy.stats as ss
import pandas as pd
import sklearn.ensemble
import sys
import os
import xgboost as xgb




with open("data_after_parse.txt", "r") as input_f:
    WHITE='PNBRQK'
    BLACK='pnbrqk'
    EMPTY='12345678'
    TURN={
        'w': 0,
        'b': 1
    }

    EN_PASSANT={
        '-': 0,
        'a3': 1,
        'b3': 2,
        'c3': 3,
        'd3': 4,
        'e3': 5,
        'f3': 6,
        'g3': 7,
        'h3': 8,
        'a6': 9,
        'b6': 10,
        'c6': 11,
        'd6': 12,
        'e6': 13,
        'f6': 14,
        'g6': 15,
        'h6': 16,
    }

    line = input_f.read().split('\n')[:-1]
    print(len(line),'!')

    X_train = np.zeros((len(line),789), dtype=object)
    Y_train = np.zeros(len(line))

    for idx, fen in enumerate(line):
        fen = fen.replace('\t', ' ')
        # print(fen.split(" "))
        try:
            pos, turn, castling, en_passant, evaluation = fen.split(" ")
        except:
            raise ValueError(f"{idx} <- {fen}")
        Board = np.zeros(789, dtype=int) # 768, 1, 4, 16
        counter=0

        # Reprezentacja jest w 789 wymiarach
        # Wymiary 0-767 <- jakie figury są na szachownicy i gdzie
        # (pierwsze 64 wymiary: białe piony; reszta wedle konwencji na górze pliku)
        # Wymiar 768 <- czyj ruch
        # Wymiary 769-772 <- roszady
        # Wymiary 773-788 <- bicia w przelocie

        for line in pos.split('/'):
            for i in line:
                if i in WHITE:
                    n=WHITE.index(i)
                    Board[64*n+counter]=1
                    counter+=1
                elif i in BLACK:
                    n=BLACK.index(i)
                    Board[64*(6+n)+counter]=1
                    counter+=1
                else:
                    counter+=int(i)

        Board[768]=TURN[turn]

        for i,s in enumerate("KkQq"):
            if s in castling:
                Board[769+i]=1

        if en_passant!='-':
            Board[772+EN_PASSANT[en_passant]]=1

        X_train[idx] = Board
        Y_train[idx] = evaluation

idxs=[]
# X_train=X_train.reshape(-1,789)

with open('numbers.txt') as f:
    n=0
    l=f.read()
    for i in l.split('\n'):
        try:
            x=int(i)
        except:
            break
        if n+x>len(X_train): break

        idxs+=[(n,n+x)]
        n+=x

np.random.shuffle(idxs)
idx1,idx2=[],[]

ratio=0.7

n=0
for i,j in idxs:
    n+=j-i
    if n>int(ratio*len(X_train)):
        idx2+=[k for k in range(i,j)]
    else:
        idx1+=[k for k in range(i,j)]

print(X_train.shape)

PCA=sklearn.decomposition.PCA(n_components=150)
# X_train=PCA.fit_transform(X_train)

print(X_train.shape)

X_tr,Y_tr=X_train[idx1],Y_train[idx1]
X_test,Y_test=X_train[idx2],Y_train[idx2]

print(len(X_tr),len(X_test))

BoostTrees = xgb.XGBRegressor(tree_method='hist',learning_rate=0.1,device='cuda',n_estimators=150000,num_parallel_tree=1,max_depth=12)

BoostTrees.fit(X_tr, Y_tr)

Ys=BoostTrees.predict(X_test)

print(f"Score: {BoostTrees.score(list(X_train),Y_train)}")
print(f"MSE: {sklearn.metrics.mean_squared_error(Ys,Y_test)}")

67854 !
(67854, 789)
(67854, 789)
47491 20278


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:07:00] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


Score: 0.7172635932027345
MSE: 0.053270142093051684


Przeprowadziliśmy serie testów starając się najlepiej dopasować parametry

In [ ]:

#!/usr/bin/python3

# XGBoost na naszych szachach

import itertools
import numpy as np
import scipy.stats as ss
import pandas as pd
import sklearn.ensemble
import sys
import os
import xgboost as xgb
import time


with open(sys.argv[1],"r") as input_f:
    WHITE='PNBRQK'
    BLACK='pnbrqk'
    EMPTY='12345678'
    TURN={
        'w': 0,
        'b': 1
    }

    EN_PASSANT={
        '-': 0,
        'a3': 1,
        'b3': 2,
        'c3': 3,
        'd3': 4,
        'e3': 5,
        'f3': 6,
        'g3': 7,
        'h3': 8,
        'a6': 9,
        'b6': 10,
        'c6': 11,
        'd6': 12,
        'e6': 13,
        'f6': 14,
        'g6': 15,
        'h6': 16,
    }

    line = input_f.read().split('\n')[:-1]
    print(len(line),'!')

    X_train = np.zeros((len(line),789))
    Y_train = np.zeros(len(line))
    
    for idx, fen in enumerate(line):
        fen = fen.replace('\t', ' ')
        # print(fen.split(" "))
        try:
            pos, turn, castling, en_passant, evaluation = fen.split(" ")
        except:
            raise ValueError(f"{idx} <- {fen}")
        Board = np.zeros(789, dtype=int) # 768, 1, 4, 16
        counter=0

        # Reprezentacja jest w 789 wymiarach 
        # Wymiary 0-767 <- jakie figury są na szachownicy i gdzie 
        # (pierwsze 64 wymiary: białe piony; reszta wedle konwencji na górze pliku)
        # Wymiar 768 <- czyj ruch
        # Wymiary 769-772 <- roszady
        # Wymiary 773-788 <- bicia w przelocie
        
        for line in pos.split('/'):
            for i in line:
                if i in WHITE:
                    n=WHITE.index(i)
                    Board[64*n+counter]=1
                    counter+=1
                elif i in BLACK:
                    n=BLACK.index(i)
                    Board[64*(6+n)+counter]=1
                    counter+=1
                else:
                    counter+=int(i)

        Board[768]=TURN[turn]

        for i,s in enumerate("KkQq"):
            if s in castling:
                Board[769+i]=1

        if en_passant!='-':
            Board[772+EN_PASSANT[en_passant]]=1

        X_train[idx] = Board
        Y_train[idx] = evaluation

idxs=[]
# X_train=X_train.reshape(-1,789)

with open('numbers.txt') as f:
    n=0
    l=f.read()
    for i in l.split('\n'):
        try:
            x=int(i)
        except:
            break
        if n+x>len(X_train): break

        idxs+=[(n,n+x)]
        n+=x

np.random.shuffle(idxs)
idx1,idx2=[],[]

ratio=0.8

n=0
for i,j in idxs:
    n+=j-i
    if n>int(ratio*len(X_train)): 
        idx2+=[k for k in range(i,j)]
    else:
        idx1+=[k for k in range(i,j)]

print(X_train.shape)

# Tu w(y)łączamy PCA
PCA=sklearn.decomposition.PCA(n_components=100)
# X_train=PCA.fit_transform(X_train)

print(X_train.shape)

X_tr,Y_tr=X_train[idx1],Y_train[idx1]
X_test,Y_test=X_train[idx2],Y_train[idx2]

print(len(X_tr),len(X_test))

# Testowane wartości
l1_vals=[0,0.01,0.02,0.05,0.1,0.2,0.5,1,2,5,10]
l2_vals=[0,0.01,0.02,0.05,0.1,0.2,0.5,1,2,5,10]
depth_vals=range(3,13)
ests=range(5000,45000,5000)

min_mse=1
p=[]
for l1,l2,d,n in itertools.product(l1_vals,l2_vals,depth_vals,ests):
    print(f"\nl1={l1},l2={l2},d={d},n={n}")

    time1=time.time()

    BoostTrees = xgb.XGBRegressor(tree_method='hist',learning_rate=0.1,device='cuda',n_estimators=n,num_parallel_tree=1,max_depth=d,reg_alpha=l1,reg_lambda=l2)
    BoostTrees.fit(X_tr, Y_tr)
    Ys=BoostTrees.predict(X_test)
    mse=sklearn.metrics.mean_squared_error(Ys,Y_test)
    time2=time.time()

    print(f"Score: {BoostTrees.score(list(X_train),Y_train)}")
    print(f"MSE: {mse}")
    print("Time:",time2-time1)

    if mse<min_mse:
        min_mse=mse
        p=[l1,l2,d,n]

print(f"\nMinimum MSE={min_mse} for l1={p[0]},l2={p[1]},d={p[2]},n={p[3]}.")
